In [8]:
import calendar
import numpy as np
import pandas as pd
import string
import os
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# setting Sunday as the first day of the week
calendar.setfirstweekday(6)

def get_week_of_month(year, month, day):
    """
    Get the week of the month for a given date.

    Args:
        year (int): The year of the date.
        month (int): The month of the date.
        day (int): The day of the date.

    Returns:
        str: The week of the month in Chinese characters.
    """
    x = np.array(calendar.monthcalendar(year, month))
    week_of_month = np.where(x==day)[0][0] + 1
    
    # Define a dictionary to map numeric values to Chinese characters
    chinese_weeks = {
        1: "第一周",
        2: "第二周",
        3: "第三周",
        4: "第四周",
        5: "第五周"
    }
    
    return chinese_weeks.get(week_of_month, "Invalid week")

def create_working_days_list(year, month, national_holidays):
    """
    Create a list of working days with their corresponding week number.

    Args:
        year (int): The year.
        month (int): The month.
        national_holidays (list): A list of national holidays.

    Returns:
        list: A list of dictionaries containing the working days and their week numbers.
    """
    # Get the number of days in the month
    _, days = calendar.monthrange(year, month)
    
    # Create a list of all days in the month
    days_list = [f"{year}/{month}/{day}" for day in range(1, days+1)]
    
    working_days_list = []
    # Iterate over each day in the month
    for day in range(1, days+1):
        # Check if the day is a working day (Monday to Friday) and not a national holiday
        if calendar.weekday(year, month, day) < 5 and f"{year}/{month}/{day}" not in national_holidays:
            # Add the working day and its corresponding week number to the list
            working_days_list.append({
                "Date": f"{year}/{month}/{day}",
                "Week": get_week_of_month(year, month, day)})
    
    # Return the list of working days and the list of all days in the month
    return working_days_list, days_list

def process_data(year, month, national_holidays):
    """
    Process data for a given year, month, and list of national holidays.

    Parameters:
    - year (int): The year for which the data is processed.
    - month (int): The month for which the data is processed.
    - national_holidays (list): A list of national holidays.

    Returns:
    - df (DataFrame): The processed data.
    - df_date (DataFrame): The data with dates.

    """
    # Read the IT data from the 'IT.xlsx' file
    df_it = pd.read_excel('./IT.xlsx')
    cnt_people = len(df_it) + 2
    
    # Create a list of working days and a list of all days in the month
    working_days_list, days_list = create_working_days_list(year, month, national_holidays)
    
    # Create a DataFrame from the list of working_days_list & days_list
    df = pd.DataFrame(working_days_list) 
    df_date = pd.DataFrame({'Date': days_list})
    
    # Join the working days DataFrame with the dates DataFrame based on the 'Date' column
    df_date = df_date.join(df.set_index('Date'), on='Date')
    
    # Transpose the working days DataFrame and set the 'Week' column as the index
    df = df.set_index('Week').T
    
    # Define the order of columns in the final DataFrame
    order_list = ['Date']
    order_list.extend(list(range(0, 69)))
    
    # Concatenate the IT data DataFrame with the working days DataFrame and reindex the columns
    df = pd.concat([df_it, df], axis=1).reindex(order_list)
    
    # Return the processed data DataFrame and the data with dates DataFrame
    return df, df_date, cnt_people

def append_dataframe_to_excel(df, file_path, sheet_name):
    """
    Appends a pandas DataFrame to an existing Excel file or creates a new file if it doesn't exist.

    Args:
        df (pandas.DataFrame): The DataFrame to be appended to the Excel file.
        file_path (str): The path of the Excel file.
        sheet_name (str): The name of the sheet in the Excel file.

    Returns:
        None
    """
    # Check if the file exists
    if not os.path.isfile(file_path):
        with pd.ExcelWriter(file_path, mode='w', engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Append the dataframe to the Excel file with a sheet name based on the month
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists="replace") as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

def merge_excel_cells(file_path, sheet_name):
    """
    Merges cells with the same value in each row of a specified sheet in an Excel file.

    Args:
        file_path (str): The path of the Excel file.
        sheet_name (str): The name of the sheet to perform the cell merging.
    """
    # Load the workbook
    book = load_workbook(file_path)
    # Get the specified sheet
    sheet = book[sheet_name]
    
    # Iterate through each row in the sheet
    for row in sheet.iter_rows():
        prev_value = None
        merge_start = None
        
        # Iterate through each cell in the row
        for cell in row:
            # Check if the cell value is different from the previous cell value
            if cell.value != prev_value:
                # If there is a merge in progress, end it and merge the cells
                if merge_start is not None:
                    merge_end = cell.offset(column=-1)
                    sheet.merge_cells(start_row=merge_start.row, start_column=merge_start.column,
                                      end_row=merge_end.row, end_column=merge_end.column)
                # Start a new merge
                merge_start = cell
            # Update the previous cell value
            prev_value = cell.value
        
        # If there is a merge in progress at the end of the row, end it and merge the cells
        if merge_start is not None:
            merge_end = row[-1]
            sheet.merge_cells(start_row=merge_start.row, start_column=merge_start.column,
                              end_row=merge_end.row, end_column=merge_end.column)
    
    # Save the modified workbook
    book.save(file_path)

def create_excel_func(df_date, sheet_name_, cnt_people):
    """
    Creates an Excel sheet with calculated values based on the given data.

    Args:
        df_date (pandas.DataFrame): The input data frame.
        sheet_name_ (str): The name of the sheet.
        cnt_people (int): The total number of people.

    Returns:
        pandas.DataFrame: The updated data frame with calculated values.
    """
    # Convert the data frame to a list
    data = df_date.values.tolist()
    i = 0
    for idx, row in enumerate(data):
        column_letter = get_column_letter(6 + i)
        cell_range = f'${column_letter}$3:${column_letter}${cnt_people}'
        cnt_IT_total = f'''=COUNTA('{sheet_name_}'!$E$3:$E${cnt_people})'''
        if type(row[1]) == str:
            cnt_WFH= f'''=COUNTIF('{sheet_name_}'!{cell_range},"居家工作")'''
            pct_WFH = f'''ROUND(([@居家工作]/[@IT總人數])*100,2) & "%"'''
            cnt_office_work = f'''=COUNTIF('{sheet_name_}'!{cell_range},"進公司") + COUNTIF('{sheet_name_}'!{cell_range},"出差")'''
            pct_office_work = f'''ROUND(([@進公司]/[@IT總人數])*100,2) & "%"'''
            cnt_leave = f'''=COUNTIF('{sheet_name_}'!{cell_range},"*假")'''
            pct_leave = f'''ROUND(([@請假]/[@IT總人數])*100,2) & "%"'''
            cnt_empty = f'''=COUNTIF('{sheet_name_}'!{cell_range},"")'''
            pct_empty = f'''ROUND(([@未填]/[@IT總人數])*100,2) & "%"'''
            cnt_filled = f'''=B{idx+2}+D{idx+2}+F{idx+2}'''
            pct_filled = f'''ROUND(([@已填]/[@IT總人數])*100,2) & "%"'''
            check_cnt = f'''=IF(($H{idx+2}+$J{idx+2})=$L{idx+2},"OK", "error")'''
            # Append the calculated values to the data list
            data[idx].extend([
                cnt_WFH, pct_WFH, cnt_office_work, pct_office_work, cnt_leave, pct_leave,
                cnt_empty, pct_empty, cnt_filled, pct_filled, cnt_IT_total, check_cnt
                ])
            i += 1
        else:
            # If the second column is not a string, fill the row with '-' values
            data[idx].extend(['-' for _ in range(10)])
            data[idx].append(cnt_IT_total)

    # Create a new data frame with the updated values
    df_date = pd.DataFrame(data, columns=['Date', 'Week', '居家工作', '居家工作%', '進公司', '進公司%', '請假', '請假%', '未填', '未填%', '已填', '已填%', 'IT總人數','Column2'])
    # Drop the 'Week' column from the data frame
    df_date = df_date.drop('Week', axis=1)
    return df_date

def process_and_save_data(year, month, national_holidays):
    """
    Process the data for the specified year, month, and list of national holidays,
    and save the processed data to an Excel file.

    Args:
        year (int): The year.
        month (int): The month.
        national_holidays (list): A list of national holidays.

    Returns:
        None
    """
    # Set the file path and sheet name
    file_path = f'./output_{year}_{str(month).zfill(2)}.xlsx'
    sheet_name_ = f'{year}_{str(month).zfill(2)}'

    # Process the data
    df, df_date, cnt_people = process_data(year, month, national_holidays)

    # Append the processed data to the Excel file
    append_dataframe_to_excel(df, file_path, sheet_name_)

    # Create the Excel function sheet
    df_date = create_excel_func(df_date, sheet_name_, cnt_people)

    # Merge cells in the Excel file
    merge_excel_cells(file_path, sheet_name_)

    # Save the Excel function sheet
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists="replace") as writer:
        df_date.to_excel(writer, sheet_name=f'{sheet_name_}_Func', index=False)

# Set the year and month
year = int(input("請輸入年份: "))
month = int(input("請輸入月份: "))

# Define the list of national holidays
national_holidays = [
    "2024/1/1", "2024/2/8", "2024/2/9", "2024/2/10", "2024/2/11", "2024/2/12", "2024/2/13", "2024/2/14", "2024/2/13", "2024/2/28",
    "2024/4/4", "2024/4/5", "2024/5/1", "2024/6/10", "2024/9/16", "2024/9/17", "2024/10/10", "2024/10/11"
]

process_and_save_data(year, month, national_holidays)